In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import missingno  as msno
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, median_absolute_error, r2_score, mean_absolute_error
from sklearn.feature_selection import SelectKBest, SelectPercentile, f_regression, f_classif

In [2]:
portogrande = pd.read_csv('../data/portogrande.txt')

FileNotFoundError: [Errno 2] File ../data/portogrande.txt does not exist: '../data/portogrande.txt'

In [ ]:
portogrande.columns

In [ ]:
portogrande.info()

# Tratando dados Faltantes

In [ ]:
# tabela percentual de dados faltantes
def tabela_faltantes():
    features_numericas = ["hora","temp_ins", "temp_max", "temp_min", "umid_inst","umid_max","umid_min",
                          "pto_orvalho_inst", "pto_orvalho_max", "pto_orvalho_min","pressao","pressao_max",
                          "pressao_min","vento_vel", "vento_direcao", "vento_rajada", "radiacao","precipitacao"]

    df = portogrande[features_numericas]


    total = df.isnull().sum().sort_values(ascending=False)
    percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)


    missing_data = pd.concat([total, percent], axis=1,join='outer', keys=['Dados Ausentes', '% Percentual'])
    missing_data.index.name =' Variaveis numericas'
    return missing_data

In [ ]:
tabela_faltantes()

In [ ]:
# Preenchendo as colunas com o valor da média:
mean_vra = portogrande.vento_rajada.mean()
mean_orv_min = portogrande.pto_orvalho_min.mean()
mean_orv_max = portogrande.pto_orvalho_max.mean()
mean_umid_min = portogrande.umid_min.mean()
mean_umid_max = portogrande.umid_max.mean()
mean_temp_min = portogrande.temp_min.mean()
mean_temp_max = portogrande.temp_max.mean()
mean_pressao_min = portogrande.pressao_min.mean()
mean_pressao_max = portogrande.pressao_max.mean()

portogrande['vento_rajada'] = portogrande['vento_rajada'].fillna(mean_vra)
portogrande['pto_orvalho_min'] = portogrande['pto_orvalho_min'].fillna(mean_orv_min)
portogrande['pto_orvalho_max'] = portogrande['pto_orvalho_max'].fillna(mean_orv_max)
portogrande['umid_min'] = portogrande['umid_min'].fillna(mean_umid_min)
portogrande['umid_max'] = portogrande['umid_max'].fillna(mean_umid_max)
portogrande['temp_min'] = portogrande['temp_min'].fillna(mean_temp_min)
portogrande['temp_max'] = portogrande['temp_max'].fillna(mean_temp_max)
portogrande['pressao_min'] = portogrande['pressao_min'].fillna(mean_pressao_min)
portogrande['pressao_max'] = portogrande['pressao_max'].fillna(mean_pressao_max)

In [ ]:
tabela_faltantes() # vamos verificar os dados depois de tratarmos os dados ausentes

- Matriz de nulidade

In [ ]:
msno.matrix(portogrande)

## Distribuição dos Dados

- Precipitação 

In [ ]:
sns.distplot(portogrande.precipitacao)

- Distribuição da Umidade relativa do ar

In [ ]:
sns.distplot(portogrande.umid_inst)
sns.distplot(portogrande.umid_max)
sns.distplot(portogrande.umid_min)
plt.legend(labels=['Umidade instante','Umidade Max','Umidade min'], ncol=3, loc='upper left');

- Distribuição da temperatura

In [ ]:
sns.distplot(portogrande.temp_ins)
sns.distplot(portogrande.temp_max)
sns.distplot(portogrande.temp_min)
plt.legend(labels=['Temp inst','Temp Max','Temp Min'], ncol=3, loc='upper left');

-  Orvalho 

In [ ]:
sns.distplot(portogrande.pto_orvalho_inst)
sns.distplot(portogrande.pto_orvalho_max)
sns.distplot(portogrande.pto_orvalho_min)
plt.legend(labels=['Orv inst','Orv Max','Orv Min'], ncol=3, loc='upper left');

- Pressão Atmosferica

In [ ]:
sns.distplot(portogrande.pressao)
sns.distplot(portogrande.pressao_max)
sns.distplot(portogrande.pressao_min)
plt.legend(labels=['Pressão','Pressão Max','Pressão Min'], ncol=3, loc='upper left');

OBS:. Apos te feito alguns testes de hipotese para normalidade (na verdade fiz na EDA só pra precipitação), heatcolor de correlação e toda analise percebeu-se que as features acima são as mais importantes  

## Seleção de Features 

- Selecionando o nosso X e Y  

In [ ]:
y = portogrande['temp_ins'] # temperatura no instante
X = portogrande.drop(['temp_ins','codigo','data','hora'], axis=1) # removendo temperatura no instante e algumas features 

- Separando em Treino e Teste

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.7, random_state=0)
Xtrain.shape, Xtest.shape, ytrain.shape, ytest.shape 

In [ ]:
k_vs_score = []

for k in range(2,17,2): 
    selector = SelectKBest(score_func=f_regression, k=5)

    Xtrain2 = selector.fit_transform(Xtrain, ytrain)
    Xtest2 = selector.transform(Xtest)

    mdl = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=0)
    mdl.fit(Xtrain2, ytrain)

    p = mdl.predict(Xval2)

    score = mean_absolute_error(ytest, p)
    print(f'k = {k} - MAE = {score}')

    k_vs_score.append(score)

In [ ]:
pd.Series(k_vs_score, index=range(2,17,2)).plot(figsize=(10,7)) # vamos printar um grafico de frequencia

In [ ]:
selector = SelectKBest(score_func=f_regression, k=8) # selecionamos o intervalo
selector.fit(Xtrain, ytrain)

In [ ]:
mask = selector.get_support()
Xtest.columns[mask] # vamos ver as features

In [ ]:
pd.Series(selector.scores_, index=Xtrain.columns).sort_values().tail().plot.barh()

In [ ]:
Xtrain.columns

In [ ]:
features = ['temp_max', 'temp_min', 'umid_inst', 'umid_max', 'umid_min'] # selecionando as variveis mais importantes
X_train = Xtrain[features]
X_teste = Xtest[features]
# y_teste = yval[features]

In [ ]:
ytest

### Vamos usar o Random Forest Regressor

In [ ]:
regressor = RandomForestRegressor( 
           criterion='mae', 
           max_depth=8,
           max_leaf_nodes=None,
           min_impurity_split=None,
           min_samples_leaf=1,
           min_samples_split=2,
           min_weight_fraction_leaf=0.0,
           n_estimators= 500,
           n_jobs=-1,
           random_state=0,
           verbose=0,
           warm_start=False
)

## Treinando  o modelo

In [ ]:
regressor.fit(X_train, ytrain)

## Realizando a predição de "Temperatura no instante" da  base teste

In [ ]:
y_pred_test = regressor.predict(X_teste)
y_pred_train = regressor.predict(X_train)

In [ ]:
print(f'𝑅² - Coeficiente de determinação: {r2_score(ytrain, y_pred_train).round(8)}') # 𝑅²
print(f'MSE - Erro médio quadrático: {mean_squared_error(ytrain, y_pred_train).round(8)}') # Mean Squared Error
print(f'MAE - Mediana dos erros de predição: {median_absolute_error(ytrain, y_pred_train).round(8)}') # Median Absolute Error